### Zadanie 1

Wykorzystując dowolny zbiór danych ze zbioru EUROSTAT, wyświetl jedną z cech numerycznych w postaci wykresu z mapą z przykładu w tym labie. Musisz dokonać mapowania nazw krajów, gdyż w parametrach wykresu oraz danych EUROSTAT gdyż te nazwy się trochę różnią. Link do pliku, który to ułatwi jest w komentarzu w kodzie przykładu wykresu z mapą.



In [2]:
import plotly.express as px
import eurostat

In [3]:
code_to_country = {
    'FR': 'France', 'DE': 'Germany', 'PL': 'Poland', 'IT': 'Italy', 'ES': 'Spain',
    'NL': 'Netherlands', 'BE': 'Belgium', 'SE': 'Sweden', 'AT': 'Austria', 'CZ': 'Czechia',
    'SK': 'Slovakia', 'HU': 'Hungary', 'DK': 'Denmark', 'PT': 'Portugal', 'FI': 'Finland',
    'NO': 'Norway', 'IE': 'Ireland', 'RO': 'Romania', 'BG': 'Bulgaria', 'HR': 'Croatia',
    'LT': 'Lithuania', 'LV': 'Latvia', 'EE': 'Estonia', 'SI': 'Slovenia', 'LU': 'Luxembourg',
    'EL': 'Greece', 'CY': 'Cyprus', 'MT': 'Malta'
}

In [4]:
df = eurostat.get_data_df('nama_10_pc')

df = df.rename(columns={'geo\\TIME_PERIOD': 'geo'})
df_long = df.melt(id_vars=['freq', 'unit', 'na_item', 'geo'], var_name='year', value_name='value')

# Step 3: Filter for latest year & GDP per capita in PPS
latest_year = df_long['year'].max()
df_filtered = df_long[
    (df_long['year'] == latest_year) &
    (df_long['unit'] == 'PPS_HAB') &
    (df_long['na_item'] == 'B1GQ')
].copy()

In [5]:
fig = px.choropleth(
    df_filtered,
    locations='geo',
    locationmode='ISO-3',
    color='value',
    color_continuous_scale="Viridis",
    range_color=(df_filtered['value'].min(), df_filtered['value'].max()),
    labels={'value': 'GDP per capita (PPS)'},
    title=f"GDP per capita (PPS) in {latest_year}"
)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0, "t":40, "l":0, "b":0})
fig.show()

### Zadanie 2

Wykorzystując zbiór wine z biblioteki sklearn (`load_wine()`), podobnie jak ze zbiorze iris) dokonaj analizy PCA tak aby:
* zachować minimum 99% wariancji
* zachować minimum 95% wariancji
* zachować minimum 90% wariancji

Ile cech pozostało w tych zbiorach?



In [6]:
from sklearn.datasets import load_wine
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

In [7]:
data = load_wine()
X = data.data

X_scaled = StandardScaler().fit_transform(X)

pca = PCA()
X_pca = pca.fit_transform(X_scaled)

cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
n_components_99 = np.argmax(cumulative_variance >= 0.99) + 1

print(f"Liczba cech zachowujących >= 90% wariancji: {n_components_90}")
print(f"Liczba cech zachowujących >= 95% wariancji: {n_components_95}")
print(f"Liczba cech zachowujących >= 99% wariancji: {n_components_99}")

Liczba cech zachowujących >= 90% wariancji: 8
Liczba cech zachowujących >= 95% wariancji: 10
Liczba cech zachowujących >= 99% wariancji: 12


### Zadanie 3

Wykorzystując przykład w labie wygeneruj wykres korelacji wartości cech wględem klasy decyzyjnej (target) pokazany w labie (scatter plot). Czy cechy, które PCA odrzuciło faktycznie były wskazane jako najlepsi kandydaci pod względem separowalności liniowej?



In [8]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import plotly.express as px

In [9]:
df = px.data.iris()
features = ["sepal_width", "sepal_length", "petal_width", "petal_length"]

In [10]:
X = df[features]
y = df["species"]

X_scaled = StandardScaler().fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

loadings = pd.DataFrame(pca.components_.T, columns=["PC1", "PC2"], index=features)

print(f"Wkład cech do komponentów PCA:\n{loadings}")

Wkład cech do komponentów PCA:
                   PC1       PC2
sepal_width  -0.263355  0.925556
sepal_length  0.522372  0.372318
petal_width   0.565611  0.065416
petal_length  0.581254  0.021095


### Zadanie 4

Wykorzystując przykład w skrypcie z wykładu numer 11 (https://github.com/agazbrzezny/MAD_SWPS_2025/blob/master/wyklad_10/redukcja_wielowym.ipynb) pokazujący użycie `SelectFromModel` (punkt 4 wykładu) wykonaj tę samą sekwencję czynności z użyciem klasyfikatora RandomForest do selekcji cech. Po przetrenowaniu modelu drzewa stwórz 3 zbiory cech z progiem jak w zadaniu 2 sumując `feature importance`. Czy zostały wybrane te same cechy co w zadaniu 2?

In [11]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np

In [12]:
iris = load_iris()
X, y = iris.data, iris.target
feature_names = iris.feature_names

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

forest = RandomForestClassifier(n_estimators=50, random_state=42)
forest.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, random_state=42)

In [14]:
importances = forest.feature_importances_
print("Feature importances:")
for name, imp in zip(feature_names, importances):
    print(f"{name}: {imp:.4f}")

Feature importances:
sepal length (cm): 0.1106
sepal width (cm): 0.0478
petal length (cm): 0.4192
petal width (cm): 0.4224


In [15]:
sorted_idx = np.argsort(importances)[::-1]
sorted_importances = importances[sorted_idx]
sorted_features = np.array(feature_names)[sorted_idx]
cumulative_importance = np.cumsum(sorted_importances)

print("\nSkumulowana suma importance:")
for i, (f, ci) in enumerate(zip(sorted_features, cumulative_importance)):
    print(f"{i+1}. {f} – suma: {ci:.4f}")


Skumulowana suma importance:
1. petal width (cm) – suma: 0.4224
2. petal length (cm) – suma: 0.8416
3. sepal length (cm) – suma: 0.9522
4. sepal width (cm) – suma: 1.0000


In [16]:
thresholds = [0.8, 0.9, 1.0]
feature_sets = {}
for thresh in thresholds:
    mask = cumulative_importance <= thresh
    if not np.any(mask) or cumulative_importance[mask].sum() < thresh:
        next_index = np.sum(mask)
        if next_index < len(feature_names):
            mask[next_index] = True
    selected = sorted_features[mask]
    feature_sets[thresh] = list(selected)

In [17]:
for thresh, feats in feature_sets.items():
    print(f"\nCechy przy progu {int(thresh*100)}% importance:")
    for f in feats:
        print(f" - {f}")


Cechy przy progu 80% importance:
 - petal width (cm)
 - petal length (cm)

Cechy przy progu 90% importance:
 - petal width (cm)
 - petal length (cm)

Cechy przy progu 100% importance:
 - petal width (cm)
 - petal length (cm)
 - sepal length (cm)
 - sepal width (cm)


In [18]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_sel', SelectFromModel(
        RandomForestClassifier(n_estimators=50, random_state=42),
        threshold='median'
    )),
    ('clf', LogisticRegression(max_iter=1000))
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_sel',
                 SelectFromModel(estimator=RandomForestClassifier(n_estimators=50,
                                                                  random_state=42),
                                 threshold='median')),
                ('clf', LogisticRegression(max_iter=1000))])

In [19]:
mask = pipeline.named_steps['feature_sel'].get_support()
selected_features = [name for name, keep in zip(feature_names, mask) if keep]

In [20]:
print(f"Cechy wybrane przez SelectFromModel (median threshold): \n{selected_features}")

Cechy wybrane przez SelectFromModel (median threshold): 
['petal length (cm)', 'petal width (cm)']


In [21]:
for thresh, feats in feature_sets.items():
    common = set(feats).intersection(set(selected_features))
    print(f"Pokrycie z progiem {int(thresh*100)}%: {len(common)} wspólnych cech")
    print(f"   Wspólne cechy: {list(common)}\n")

Pokrycie z progiem 80%: 2 wspólnych cech
   Wspólne cechy: ['petal width (cm)', 'petal length (cm)']

Pokrycie z progiem 90%: 2 wspólnych cech
   Wspólne cechy: ['petal width (cm)', 'petal length (cm)']

Pokrycie z progiem 100%: 2 wspólnych cech
   Wspólne cechy: ['petal width (cm)', 'petal length (cm)']

